### Project 6 - Exceptions

Suppose we have a Widget online sales application and we are writing the backend for it. We want a base `WidgetException` class that we will use as the base class for all our custom exceptions we raise from our Widget application.

Furthermore we have determined that we will need the following categories of exceptions:

```
1. Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
2. Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

Write an exception class hierarchy to capture this. In addition, we would like to implement the following functionality:
* implement separate internal error message and user error message
* implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
* implement a logging function that can be called to log the exception details, time it occurred, etc.
* implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

##### Bonus

Log the traceback too - you'll have to do a bit of research for that! 

I'm going to use the `TracebackException` class in the `traceback` module:

https://docs.python.org/3/library/traceback.html#tracebackexception-objects

In particular, look at the class method `from_exception` (and remember that inside your exception class, the exception will be `self`!) and the `format` instance method. That method returns a generator, so you'll need to `list` it to print out everything in that traceback.

Good luck!

In [15]:
# Imports
import datetime
import traceback
from typing import Generator
from abc import ABC, abstractmethod
from http import HTTPStatus

In [73]:
# Our base exception
class AppBaseException(Exception):
    # This class variable will be the message of the exception unless a message is supplied in the arguments
    message = "Base application exception"

    def __init__(self, *args, custom_message: str = None):
        Exception.__init__(self, args)
        if args:
            # Override the message with the first argument passed to the instanciator
            self.message = args[0]
        self.custom_message = custom_message or self.message
        # self.timestamp = datetime.utcnow().isoformat()
        self.timestamp = datetime.datetime.now(datetime.UTC).isoformat()

    @property
    def traceback(self) -> Generator[str, None, None]:
        return traceback.TracebackException.from_exception(self).format()

    @property
    def exc_type(self) -> str:
        return type(self).__name__

    def log_exception(self) -> dict:
        """ Return a formatted dict representation of the exception to be logged by logger """
        return {
            'timestamp': self.timestamp,
            'type': self.exc_type,
            'message': self.message,
            'arguments': self.args,
            'tb': ''.join(list(self.traceback))
        }

    def to_json(self) -> str:
        """ Returns a json formatted representation to be returned to the user """
        raise NotImplementedError("This method has not beeing implemented.!")
        

In [74]:
class WidgetException(AppBaseException):
    message = "An internal error exception has occurred"
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR

    def __init__(self, *args, custom_message: str = None):
        super().__init__(*args, custom_message)

    def to_json(self):
        return {
            'status_code': self.http_status.value,
            'message': '{}: {}'.format(self.http_status.phrase, self.custom_message),
            'exc_type': self.exc_type,
            'timestamp': self.timestamp
        }

In [75]:
# test widget exception
e = WidgetException(custom_message="Testing message")
print(e.log_exception())
print('----------------------------------------------')
print(e.to_json())

{'timestamp': '2024-09-08T12:20:31.348534+00:00', 'type': 'WidgetException', 'message': 'Testing message', 'arguments': (('Testing message',),), 'tb': "WidgetException: ('Testing message',)\n"}
----------------------------------------------
{'status_code': 500, 'message': 'Internal Server Error: Testing message', 'exc_type': 'WidgetException', 'timestamp': '2024-09-08T12:20:31.348534+00:00'}


In [76]:
class SupplierException(WidgetException):
    message = 'Supplier exception.'

class NotManufacturedException(SupplierException):
    message = 'Widget is no longer manufactured by supplier.'
    
class ProductionDelayedException(SupplierException):
    message = 'Widget production has been delayed by supplier.'
    
class ShippingDelayedException(SupplierException):
    message = 'Widget shipping has been delayed by supplier.'
    
class CheckoutException(WidgetException):
    message = 'Checkout exception.'
    
class InventoryException(CheckoutException):
    message = 'Checkout inventory exception.'
    
class OutOfStockException(InventoryException):
    message = 'Inventory out of stock'
    
class PricingException(CheckoutException):
    message = 'Checkout pricing exception.'
    
class InvalidCouponCodeException(PricingException):
    message = 'Invalid checkout coupon code.'
    
class CannotStackCouponException(PricingException):
    message = 'Cannot stack checkout coupon codes.'

In [77]:
# Test one of the custom exceptions
e = InvalidCouponCodeException()
print(e.log_exception())
print('----------------------------------------------')
print(e.to_json())

{'timestamp': '2024-09-08T12:20:32.588018+00:00', 'type': 'InvalidCouponCodeException', 'message': None, 'arguments': ((None,),), 'tb': 'InvalidCouponCodeException: (None,)\n'}
----------------------------------------------
{'status_code': 500, 'message': 'Internal Server Error: None', 'exc_type': 'InvalidCouponCodeException', 'timestamp': '2024-09-08T12:20:32.588018+00:00'}


In [78]:
# New exception with different http status code
class ProductNotFoundException(InventoryException):
    message = "The product you are looking for has not been found"
    http_status = HTTPStatus.NOT_FOUND

In [79]:
e = ProductNotFoundException(f"Product id {3444} does not exist in current inventory", custom_message=f"Product id {3444} does not exist")
print(e.log_exception())
print('----------------------------------------------')
print(e.to_json())

{'timestamp': '2024-09-08T12:20:33.885387+00:00', 'type': 'ProductNotFoundException', 'message': 'Product id 3444 does not exist in current inventory', 'arguments': (('Product id 3444 does not exist in current inventory', 'Product id 3444 does not exist'),), 'tb': "ProductNotFoundException: ('Product id 3444 does not exist in current inventory', 'Product id 3444 does not exist')\n"}
----------------------------------------------
{'status_code': 404, 'message': 'Not Found: Product id 3444 does not exist in current inventory', 'exc_type': 'ProductNotFoundException', 'timestamp': '2024-09-08T12:20:33.885387+00:00'}


In [80]:
# Try to_json directly from AppBaseException
e = AppBaseException()
print(e.log_exception())
print('----------------------------------------------')
print(e.to_json())

{'timestamp': '2024-09-08T12:20:34.868596+00:00', 'type': 'AppBaseException', 'message': 'Base application exception', 'arguments': ((),), 'tb': 'AppBaseException: ()\n'}
----------------------------------------------


NotImplementedError: This method has not beeing implemented.!

In [81]:
# Try to create a class that not implements or override the to_json method
class CustomExc(AppBaseException):
    message = "This is a custom exception that does not implement to_json method"

    def __init__(self, *args, custom_message: str = None):
        super().__init__(*args, custom_message)

e = CustomExc()
print(e.to_json())

NotImplementedError: This method has not beeing implemented.!